## SRJ

## Adapt GCIM - LiH

## Changed the qml.DE to qml.FermionicDE

In [1]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4
operator_check = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.62
symbols = ["Li","H"]
electrons = 4
orbitals = 12
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf")
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[3840] = 1.0 # Every molecule change, you need to change this index #3840
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the ash_excitationis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    #For generating the new state, they are adding the excitation in the exponent of the rotation, thats 
    #why we need to use qml.FermionicSE or qml.FermionicDE
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
     
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
  


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    excitationlist.extend(singles)
    print('Excitations are', excitationlist)
    #print('Going to print the single generating fns.')
    for excitation in excitationlist:
        @qml.qnode(dev)
        def superposition1():
            #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
            qml.StatePrep(hf_state, wires=range(qubits))
            #print('Single Exc going in', excitation)
            qml.SingleExcitation(theta, wires=excitation)
            return qml.state()
        super1 = superposition1()
        generatingfns.append(super1)
        #print('S-Generating functions are', super1)
    #print('After adding single gfs, len is', len(generatingfns))
    print('\n\n\n')
    #print('Printing 2s generating fns')
    for i, j in itertools.combinations(excitationlist, 2):
        if len(i) == 2 and len(j) == 2:  # Ensure both are singles
            #print(f'Applying excitations: {i} and {j}')
            @qml.qnode(dev)
            def superposition2():
                #[qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
                qml.StatePrep(hf_state, wires=range(qubits))
                qml.SingleExcitation(theta, wires=i)
                qml.SingleExcitation(theta, wires=j)
                return qml.state()                
            super2= superposition2()
            generatingfns.append(super2)
            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]


    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already checked
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    #print('Print, if this is activated - HF state')
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        print('Ham matrix is', Ham_matrix)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix


In [2]:
ash_excitation, states,eig,gs_energy, Ham_matrix = adaptvqe(adapt_it=2, e_th=1e-12)
#print(Ham_matrix)

HF state is [6.123234e-17+0.j 0.000000e+00+0.j 0.000000e+00+0.j ... 0.000000e+00+0.j
 0.000000e+00+0.j 0.000000e+00+0.j]
HF state is -7.861149428178474
Excitations are [[0, 4], [0, 6], [0, 8], [0, 10], [1, 5], [1, 7], [1, 9], [1, 11], [2, 4], [2, 6], [2, 8], [2, 10], [3, 5], [3, 7], [3, 9], [3, 11]]




The adapt iteration now is 0


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.24829589380807315 for operator a⁺(2) a⁺(3) a(10) a(11)
Highest gradient excitation is [2, 3, 10, 11]
ash_excitation is [[2, 3, 10, 11]]
The length of ash_excitation before generating matrix is 1
 0: ─╭|Ψ⟩──────────────────────────────────┤  State
 1: ─├|Ψ⟩──────────────────────────────────┤  State
 2: ─├|Ψ⟩─╭FermionicDoubleExcitation(0.79)─┤  State
 3: ─├|Ψ⟩─├FermionicDoubleExcitation(0.79)─┤  State
 4: ─├|Ψ⟩─│────────────────────────────────┤  State
 5: ─├|Ψ⟩─│────────────────────────────────┤  State
 6: ─├|Ψ⟩─│────────────────────────────────┤  State
 7: ─├|Ψ⟩─│────────────────────────────────┤  State
 8: ─├|Ψ⟩─│────────────────────────────────┤  State
 9: ─├|Ψ⟩─│────────────────────────────────┤  State
10: ─├|Ψ⟩─├FermionicDoubleExcitation(0.79)─┤  State
11: ─╰|Ψ⟩─╰FermionicDoubleExcitation(0.79)─┤  State
State count after Generating function is {'110000000011': 143, '111100000000': 857}
After adding new state, the length of states is 2
Ham matrix is [

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

The highest operator value is 0.07640684133102962 for operator a⁺(2) a⁺(3) a(5) a(10)
Highest gradient excitation is [2, 3, 5, 10]
ash_excitation is [[2, 3, 10, 11], [2, 3, 5, 10]]
The length of ash_excitation before generating matrix is 2
 0: ─╭|Ψ⟩───────────────────────────────────────────────────────────────────┤  State
 1: ─├|Ψ⟩───────────────────────────────────────────────────────────────────┤  State
 2: ─├|Ψ⟩─╭FermionicDoubleExcitation(0.79)─╭FermionicDoubleExcitation(0.79)─┤  State
 3: ─├|Ψ⟩─├FermionicDoubleExcitation(0.79)─├FermionicDoubleExcitation(0.79)─┤  State
 4: ─├|Ψ⟩─│────────────────────────────────│────────────────────────────────┤  State
 5: ─├|Ψ⟩─│────────────────────────────────├FermionicDoubleExcitation(0.79)─┤  State
 6: ─├|Ψ⟩─│────────────────────────────────├FermionicDoubleExcitation(0.79)─┤  State
 7: ─├|Ψ⟩─│────────────────────────────────├FermionicDoubleExcitation(0.79)─┤  State
 8: ─├|Ψ⟩─│────────────────────────────────├FermionicDoubleExcitation(0.79)─┤  S

In [4]:
import pennylane as qml
import numpy as np
ash_excitation = [[2, 3, 10, 11], [0, 3, 5, 6]]
print(ash_excitation[1])
hf_state = np.array([1,1,1,1,0,0,0,0,0,0,0,0])

[0, 3, 5, 6]


In [ ]:
old_data = [-7.875951592787303, -7.875951592789975, -7.875951592833956, -7.875951592892348, -7.875951592952665, -7.87595159300971]

Bo_data = [-7.875951590107945, -7.876589700913479, -7.876589700913480, -7.876771162506847, -7.876773595926845]

# Finding difference between qml.SE and qml.FSE

In [ ]:
import pennylane as qml
import numpy as np
qubits = 4
hf_state = np.array([1,1,0,0])
theta = np.pi/4
dev = qml.device("lightning.qubit", wires=qubits)

@qml.qnode(dev)
def SE():
    qml.ash_excitationisState(hf_state, wires=range(qubits))
    qml.SingleExcitation(theta, wires=[1, 3])
    return qml.state()


ash_excitation = [[0, 2]]  # Example excitation for Single Excitation
print('Moving to FSE')
@qml.qnode(dev)
def FSE():
    qml.BasisState(hf_state, wires=range(qubits))
    qml.FermionicSingleExcitation(weight=theta, wires=[1, 2, 3])

    return qml.state()

print("Single Excitation State:", SE())
print("Fermionic Single Excitation State:", FSE())

## They are the same

In [5]:
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def stateprep(i):
    print('The excitation coming in is', i)
    qml.BasisState(hf_state, wires=range(qubits))
    if len(ash_excitation[i]) == 4:
        print('DE is ', ash_excitation[i])
        qml.DoubleExcitation(theta, wires = ash_excitation[i])
    elif len(ash_excitation[i]) == 2:
        qml.SingleExcitation(theta, wires = ash_excitation[i])
    return qml.state()


In [6]:
from pennylane.fermi import from_string

def operator(ash_excitation, theta, i):
    singles_fermi = []
    doubles_fermi = []
    doubles_pauli = []
    singles_pauli = []
    if len(ash_excitation[i]) == 4:
        print('DE is ', ash_excitation[i])
        doubles_fermi.append(from_string(f"{ash_excitation[i][3]}+ {ash_excitation[i][2]}+ {ash_excitation[i][1]}- {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}+ {ash_excitation[i][2]}- {ash_excitation[i][3]}-"))
        doubles_pauli.append(qml.jordan_wigner(doubles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * doubles_pauli[0]).operation()) #Removed the range(qubits) part
    elif len(ash_excitation[i]) == 2:
        print('SE is ', ash_excitation[i])
        singles_fermi.append(from_string(f"{ash_excitation[i][1]}+ {ash_excitation[i][0]}-")
                       - from_string(f"{ash_excitation[i][0]}+ {ash_excitation[i][1]}-"))
        singles_pauli.append(qml.jordan_wigner(singles_fermi[-1], qubits))
        k1 = qml.exp((theta/2 * singles_pauli[0]).operation()) #Removed the range(qubits) part
    exp_matrix1 = qml.matrix(k1, wire_order=range(qubits))
    print('exp matrix1', exp_matrix1.shape)
    return exp_matrix1

# Trying to add the gradient

## Updated code

In [16]:
#ash_excitation = []
Hmat = qml.matrix(H)
np.set_printoptions(threshold=np.inf)

#dev = qml.device("lightning.qubit", wires=qubits)
#@qml.qnode(dev)
def gcm_grad(ash_excitation, H, theta):

    #
    L = len(ash_excitation)
    print('Value of L',L)
    Heff = np.zeros((L,L),dtype=complex)
    ovlp = np.zeros((L,L),dtype=complex)
    d_Heff = np.zeros((L,L),dtype=complex)
    d_ovlp = np.zeros((L,L),dtype=complex)
    for i in range(L):
        for j in range(L):
            print(f'The value of i {i} and j is  {j}')
            _ = (stateprep(i).conj().T) # Creating the bra vector
            print('The bra vector is', _) 
            print('j value going in', j)
            Heff[i,j] = _.dot(Hmat.dot(stateprep(j))).real
            ovlp[i,j] = _.dot(stateprep(j)).real
            if i == L-1 and j != L-1:
                print('Case 1 is activated')
                print(f'The value of i {i} and j is  {j}')
                #d_Heff[i,j] = -1*_.dot((ash_excitation[i]@H).dot(stateprep[j]))[0,0].real
                d_Heff[i,j] = -1*_.dot((operator(ash_excitation, theta, i)@Hmat).dot(stateprep(j))).real
                d_ovlp[i,j] = _.dot((operator(ash_excitation, theta, i)).dot(stateprep(j))).real

            elif i != L-1 and j == L-1:
                print('Case 2 is activated')
                print(f'The value of i {i} and j is  {j}')
                d_Heff[i,j] = _.dot((Hmat@(operator(ash_excitation, theta, i))).dot(stateprep(j))).real
                d_ovlp[i,j] = _.dot(operator(ash_excitation, theta, i).dot(stateprep(j))).real

                
            elif i != L-1 and j != L-1:
                print('Case 3 is activated')
                print(f'The value of i {i} and j is  {j}')
                #d_Heff[i,j] = _.dot((Hmat@ash_excitation[i]-ash_excitation[i]@Hmat).dot(ash_excitation[j]@hf))[0,0].real
                d_Heff[i,j] = _.dot((Hmat@(operator(ash_excitation, theta, i))-(operator(ash_excitation, theta, i))@Hmat).dot(stateprep(j))).real
  
    #e,f = sl.eig(Heff,ovlp)
    e,f=scipy.linalg.eigh(Heff,ovlp)
    print('Eigenvalues are', e)
    idx = np.argsort(e)
    f = f[:,idx]
    H_k = np.conj(f[0]).T@Heff@f[0]
    S_k = np.conj(f[0]).T@ovlp@f[0]
    d_H_k = np.conj(f[0]).T@d_Heff@f[0]
    d_S_k = np.conj(f[0]).T@d_ovlp@f[0]
    g = (d_H_k*S_k - H_k*d_S_k)/(S_k*S_k)
    print('The value of g:',g)

In [17]:
gcm_grad(ash_excitation, H, theta)

Value of L 2
The value of i 0 and j is  0
The excitation coming in is 0
DE is  [2, 3, 10, 11]
The bra vector is [ 0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.        -0.j  0.        -0.j
  0.        -0.j  0.        -0.j  0.       

In [15]:
print(_)

## Original code

In [ ]:
#ash_excitation = []
Hmat = qml.matrix(H)
dev = qml.device("lightning.qubit", wires=qubits)
@qml.qnode(dev)
def gcm_grad(ash_excitation, H, theta):
    #
    # _,op = ans_pool(indx_tuple,rot)
    #ash_excitation = bas_old.copy()
    #ash_excitation.insert(0, sps.csc_matrix( np.eye(op_expmat.shape[0]) ) )
    #ash_excitation.append(excitations)
    #
    L = len(ash_excitation)
    print('Value of L',L)
    Heff = np.zeros((L,L),dtype=complex)
    ovlp = np.zeros((L,L),dtype=complex)
    d_Heff = np.zeros((L,L),dtype=complex)
    d_ovlp = np.zeros((L,L),dtype=complex)
    for i in range(L):
        for j in range(L):
            print(f'The value of i {i} and j is  {j}')
            _ = (stateprep(i).conj().T) # Creating the bra vector
            print('The bra vector is', _) 
            #_ = sps.csc_matrix.transpose(_)
            print('j value going in', j)
            Heff[i,j] = _.dot(Hmat.dot(stateprep(j))).real
            ovlp[i,j] = _.dot(stateprep(j)).real
            if i == L-1 and j != L-1:
                print('Case 1 is activated')
                print(f'The value of i {i} and j is  {j}')
                if len(ash_excitation[i]) == 4:
                    #d_Heff[i,j] = -1*_.dot((ash_excitation[i]@H).dot(stateprep[j]))[0,0].real
                    d_Heff[i,j] = -1*_.dot(((qml.DoubleExcitation(theta, wires =ash_excitation[i]))@Hmat).dot(stateprep(j))).real
                    d_ovlp[i,j] = _.dot((qml.DoubleExcitation(theta, wires = ash_excitation[i])).dot(stateprep(j))).real
                elif len(ash_excitation[i]) == 2:
                    d_Heff[i,j] = -1*_.dot(((qml.SingleExcitation(theta, wires =ash_excitation[i]))@Hmat).dot(stateprep(j))).real
                    d_ovlp[i,j] = _.dot((qml.SingleExcitation(theta, wires = ash_excitation[i])).dot(stateprep(j))).real
            elif i != L-1 and j == L-1:
                print('Case 2 is activated')
                print(f'The value of i {i} and j is  {j}')
                if len(ash_excitation[i]) == 4:
                    #d_Heff[i,j] = _.dot((Hmat@ash_excitation[i]).dot(ash_excitation[j]@hf))[0,0].real
                    d_Heff[i,j] = _.dot((Hmat@(qml.DoubleExcitation(theta, wires = ash_excitation[i]))).dot(stateprep(j))).real
                    d_ovlp[i,j] = _.dot(qml.DoubleExcitation(theta, wires=ash_excitation[i]).dot(stateprep(j))).real
                elif len(ash_excitation[i]) == 2:
                    d_Heff[i,j] = _.dot((Hmat@(qml.SingleExcitation(theta, wires=ash_excitation[i]))).dot(stateprep(j))).real
                    d_ovlp[i,j] = _.dot((qml.SingleExcitation(theta, wires=ash_excitation[i])).dot(stateprep(j))).real
                
            elif i != L-1 and j != L-1:
                print('Case 3 is activated')
                print(f'The value of i {i} and j is  {j}')
                if len(ash_excitation[i]) == 4:
                    #d_Heff[i,j] = _.dot((Hmat@ash_excitation[i]-ash_excitation[i]@Hmat).dot(ash_excitation[j]@hf))[0,0].real
                    d_Heff[i,j] = _.dot((Hmat@(qml.DoubleExcitation(theta, wires=ash_excitation[i]))-(qml.DoubleExcitation(theta, wires=i))@Hmat).dot(stateprep(j))).real
                elif len(ash_excitation[i]) == 2:
                    _.dot((Hmat@(qml.SingleExcitation(theta, wires=ash_excitation[i]))-(qml.SingleExcitation(theta, wires=ash_excitation[i]))@Hmat).dot(stateprep(j))).real
    #e,f = sl.eig(Heff,ovlp)
    e,f=scipy.linalg.eigh(Heff,ovlp)
    print('Eigenvalues are', e)

In [ ]:
gcm_grad(ash_excitation, H, theta)

## Trying to mimic an operator

In [3]:
H
Ham_matrix = qml.matrix(H)
print(Ham_matrix)

[[ 0.97995787+0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.02556571+0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.02556571+0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 ...
 [ 0.        +0.j  0.        +0.j  0.        +0.j ... -0.88210221+0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
  -0.88210221+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  1.60107877+0.j]]


In [13]:
from pennylane.fermi import from_string
electrons = 4
qubits = 12
singles, doubles = qchem.excitations(electrons, qubits)

singles_fermi = []
for ex in singles:
    #print('The ex1 is', ex)
    singles_fermi.append(from_string(f"{ex[1]}+ {ex[0]}-") - from_string(f"{ex[0]}+ {ex[1]}-"))

#print(singles_fermi)
doubles_fermi = []
excitation = [[2,3,10,11]]
for ex in excitation:
    #print('The ex2 is', ex)
    doubles_fermi.append(from_string(f"{ex[3]}+ {ex[2]}+ {ex[1]}- {ex[0]}-")
                       - from_string(f"{ex[0]}+ {ex[1]}+ {ex[2]}- {ex[3]}-"))
#print(doubles_fermi)


doubles_pauli = []
for op in doubles_fermi:
    print('Exc is ',op)
    doubles_pauli.append(qml.jordan_wigner(op, qubits))

#print(doubles_pauli[0])
theta = np.pi/4


#for i, excitation in enumerate(doubles_pauli):
k = qml.exp((doubles_pauli[0] * theta).operation())
print('The exponent is', k)
exp_matrix = qml.matrix(k, wire_order=range(qubits))
j = (Ham_matrix @ exp_matrix)

print(j)

Exc is  1.0 * a⁺(11) a⁺(10) a(3) a(2)
+ -1.0 * a⁺(2) a⁺(3) a(10) a(11)
0.125j * X(2) @ X(3) @ Y(10) @ X(11)
+ -0.125j * Y(2) @ Y(3) @ Y(10) @ X(11)
+ -0.125j * Y(2) @ X(3) @ X(10) @ X(11)
+ -0.125j * X(2) @ Y(3) @ X(10) @ X(11)
+ 0.125j * Y(2) @ X(3) @ Y(10) @ Y(11)
+ 0.125j * X(2) @ Y(3) @ Y(10) @ Y(11)
+ 0.125j * X(2) @ X(3) @ X(10) @ Y(11)
+ -0.125j * Y(2) @ Y(3) @ X(10) @ Y(11)
The exponent is Exp(1 0.09817477042468103j * (X(2) @ X(3) @ Y(10) @ X(11)) + -0.09817477042468103j * (Y(2) @ Y(3) @ Y(10) @ X(11)) + -0.09817477042468103j * (Y(2) @ X(3) @ X(10) @ X(11)) + -0.09817477042468103j * (X(2) @ Y(3) @ X(10) @ X(11)) + 0.09817477042468103j * (Y(2) @ X(3) @ Y(10) @ Y(11)) + 0.09817477042468103j * (X(2) @ Y(3) @ Y(10) @ Y(11)) + 0.09817477042468103j * (X(2) @ X(3) @ X(10) @ Y(11)) + -0.09817477042468103j * (Y(2) @ Y(3) @ X(10) @ Y(11)))
[[ 0.97995787+0.j  0.        +0.j  0.        +0.j ...  0.        +0.j
   0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.02556571+0.j  0.        

## Trying to check the code is wokring

In [ ]:
import pennylane as qml
import numpy as np

from pennylane import qchem

